# Insurance Data Cleaning Notebook

This notebook walks through each step of the data cleaning process using modular functions from `cleaning.py` and `data_loader.py`.

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from data_loader import load_data, save_data
from cleaning import (
    handle_missing_values,
    correct_data_types,
    remove_duplicates,
    handle_outliers,
    standardize_categorical
)

RAW_PATH = "../data/raw/MachineLearningRating_v3.txt"
df = load_data(RAW_PATH)
df.head()


c:\Users\tsion\insurance-risk-analytics\src\data_loader.py:9: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='|', encoding='utf-8')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


## Step 1: Handle Missing Values

We fill in missing values using suitable strategies depending on the column type and proportion of missingness.


In [18]:
df = handle_missing_values(df)
df.isna().sum().sort_values(ascending=False).head()

c:\Users\tsion\insurance-risk-analytics\src\cleaning.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown' if df[col].dtype == 'object' else 0, inplace=True)
c:\Users\tsion\insurance-risk-analytics\src\cleaning.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df[col].fillna('Unknown' if df[col].dtype == 'object' else 0

UnderwrittenCoverID    0
PolicyID               0
TransactionMonth       0
IsVATRegistered        0
Citizenship            0
dtype: int64

## Step 2: Correct Data Types

Convert columns like dates, numbers, and categories to their correct data types for accurate analysis.


In [3]:
df = correct_data_types(df)
df.dtypes.head(10)


UnderwrittenCoverID             int64
PolicyID                        int64
TransactionMonth       datetime64[ns]
IsVATRegistered                  bool
Citizenship                    object
LegalType                      object
Title                          object
Language                       object
Bank                           object
AccountType                    object
dtype: object

## checking for duplicates

In [5]:
print(df.duplicated().sum())

0


## Step 3a: Visualize Outliers in All Numeric Columns

We generate and save boxplots for all numeric columns in the dataset to assess the presence of outliers.



In [8]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Create output directory if it doesn't exist
boxplot_dir = "../outputs/boxplots"
os.makedirs(boxplot_dir, exist_ok=True)

# Select all numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Generate and save boxplots
for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col].dropna())
    plt.title(f"Boxplot of {col}")
    plt.tight_layout()
    plot_path = os.path.join(boxplot_dir, f"{col}_boxplot.png")
    plt.savefig(plot_path)
    plt.close()  # Close the plot to free memory


## Step 3b: Handle Outliers

After reviewing the boxplots, we apply the IQR method to cap outliers in all numeric columns.


In [9]:
df = handle_outliers(df)


### Comparing Value Ranges Before and After Outlier Capping
We compare the min and max values of selected numeric columns to observe the effect of the outlier treatment.


In [14]:
# Create a copy of the original DataFrame
df_before = df.copy()

# Automatically find all numeric columns
numeric_cols = df_before.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Get value ranges before outlier handling
ranges_before = df_before[numeric_cols].agg(['min', 'max']).T
ranges_before.columns = ['Before_Min', 'Before_Max']

# Apply outlier handling
df_after = handle_outliers(df_before)

# Get value ranges after outlier handling
ranges_after = df_after[numeric_cols].agg(['min', 'max']).T
ranges_after.columns = ['After_Min', 'After_Max']

# Merge the two DataFrames for side-by-side comparison
range_comparison = pd.concat([ranges_before, ranges_after], axis=1)

# Display the comparison
range_comparison


,Before_Min,Before_Max,After_Min,After_Max
UnderwrittenCoverID,6.000000e+00,3.011700e+05,6.000000e+00,3.011700e+05
PolicyID,1.400000e+01,2.324600e+04,1.400000e+01,2.324600e+04
PostalCode,1.000000e+00,9.870000e+03,1.000000e+00,9.870000e+03
mmcode,4.041200e+06,6.503072e+07,4.041200e+06,6.503072e+07
RegistrationYear,1.987000e+03,2.015000e+03,1.987000e+03,2.015000e+03
Cylinders,0.000000e+00,1.000000e+01,0.000000e+00,1.000000e+01
cubiccapacity,0.000000e+00,1.288000e+04,0.000000e+00,1.288000e+04
kilowatts,0.000000e+00,3.090000e+02,0.000000e+00,3.090000e+02
NumberOfDoors,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00
CustomValueEstimate,0.000000e+00,1.720000e+05,0.000000e+00,1.720000e+05


## Step 4: Standardize Categorical Variables

Normalize strings, strip whitespace, fix capitalization, and replace codes with human-readable values.


In [16]:
df = standardize_categorical(df)
print(df.head())

    UnderwrittenCoverID  PolicyID TransactionMonth  IsVATRegistered  \
0                145249     12827              NaT             True   
12               127439     10874              NaT             True   
18               120087     10173              NaT             True   
24                82059      7174              NaT             True   
44               127850     10902              NaT             True   

   Citizenship          LegalType Title Language                 Bank  \
0               Close Corporation    Mr  English  First National Bank   
12              Close Corporation    Mr  English  First National Bank   
18              Close Corporation    Mr  English  First National Bank   
24              Close Corporation    Mr  English  First National Bank   
44              Close Corporation    Mr  English  First National Bank   

        AccountType  ...         ExcessSelected              CoverCategory  \
0   Current account  ...  Mobility - Windscreen         

## Final Step: Save the Cleaned Dataset

Save the cleaned DataFrame to the `processed/` directory for downstream analysis or modeling.


In [17]:
save_data(df, '../data/processed/MachineLearningRating_v3_cleaned.txt')


Data saved successfully to ../data/processed/MachineLearningRating_v3_cleaned.txt.


True